In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import StackingRegressor
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
def addFeatures(df):
    
    df['YrBltAndRemod'] = df['YearBuilt'] + df['YearRemodAdd']
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['Total_sqr_footage'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['Total_Bathrooms'] = df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])
    df['Total_porch_sf'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF']

    return df

def preprocess(df):

    df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x : x.mean())    
    
    for col in df.select_dtypes(exclude='number').columns:
        df[col] = df[col].fillna('None')

    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].fillna(0)

    return df

def encoding(df):
    ohe = OneHotEncoder(sparse_output = False, drop = 'first')
    catagorical_cols = list(df.select_dtypes(exclude='number').columns)
    numaric_cols = list(df.select_dtypes(include='number').columns)
 
    encoded_values = ohe.fit_transform(df[catagorical_cols])
    encoded_cols = list(ohe.get_feature_names_out())
    df[encoded_cols] = encoded_values


    return df[encoded_cols + numaric_cols]

In [4]:
all_df = pd.concat([train_df, test_df])

index = train_df.shape[0]
target = all_df.iloc[:index]['SalePrice']

all_df = addFeatures(all_df)
all_df = preprocess(all_df)
all_df = encoding(all_df)
id = all_df['Id']
all_df.drop(columns=['Id', 'SalePrice'], inplace= True)

/tmp/ipykernel_17/400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[encoded_cols] = encoded_values
/tmp/ipykernel_17/400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[encoded_cols] = encoded_values
/tmp/ipykernel_17/400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.

In [5]:
xgb_params = {'max_depth': 4, 'learning_rate': 0.00875, 'n_estimators': 3515, 'min_child_weight': 2, 'colsample_bytree': 0.2050378195385253, 'subsample': 0.40369887914955715, 'reg_alpha': 0.3301567121037565, 'reg_lambda': 0.046181862052743}

xgbr = xgb.XGBRegressor(**xgb_params, random_state=42)

model = [
    ('ridge', Ridge(alpha=15)),
    ('XGB', xgbr),
] 

stack = StackingRegressor(estimators=model, final_estimator=LinearRegression())

stack.fit(all_df.iloc[:index], np.log1p(target.values))

StackingRegressor(estimators=[('ridge', Ridge(alpha=15)),
                              ('XGB',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.2050378195385253,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.00875, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None, min_child_weight=2,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=3515, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                  final_estimator=LinearRegression())

In [6]:
predict = stack.predict(all_df)

output = pd.DataFrame({
    'Id' : id[index:],
    'SalePrice' : np.exp(predict[index:])
})

output.to_csv('submission.csv' ,index=False)